## This notebook chooses AutoML best model and based on its predictions on validation set, it calculates a SIGMA MAD validation metric.

In [25]:
%load_ext autoreload
%autoreload 2
import os 
import sys
import time
import h2o

import pandas as pd
import numpy as np
from h2o.automl import H2OAutoML

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
module_path = os.path.abspath(os.path.join('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.modeling.functions_autoML import gen_predictions
from src.modeling.evaluation import get_metrics_from_batchs

In [3]:
pd.set_option("display.max_rows", 6)

In [4]:
df = pd.read_pickle("../../data/structured/df.pkl")
#IA Filter
df = df.loc[df.type_bool == True]

#### Getting h2o best result and making a prediction to list

In [8]:
h2o.init(ip="localhost", port=54323)
best_model = h2o.import_mojo('../../models/mojo_50_ensemble/StackedEnsemble_AllModels_AutoML_20201223_115349.zip')

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,1 min 11 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_felipe_1jtpxa
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.717 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1621206572488_4

No model summary for this model

ModelMetricsRegressionGeneric: generic
** Reported on train data. **

MSE: 0.0029843621318644473
RMSE: 0.0546293156818246
MAE: 0.04095889497837518
RMSLE: 0.030993632001274172
Mean Residual Deviance: 0.0029843621318644473

ModelMetricsRegressionGeneric: generic
** Reported on cross-validation data. **

MSE: 0.007612769296751577
RMSE: 0.0872511850736228
MAE: 0.06616340247814928
RMSLE: 0.05020608420962501
Mean Residual Deviance: 0.007612769296751577



## Predicting values based on data used to validation

In [9]:
preds, len_train, len_validation = gen_predictions(df, best_model)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
generic prediction progress: |████████████████████████████████████████████| 100%

 2550 train/test objects  
 and 2528 validation objects


## Creates new column on df. It will be used for SIGMA MAD calculus.

In [10]:
df_sigma_mad = df.iloc[-len_validation:]
df_sigma_mad['zphot'] = preds.as_data_frame()['predict'].tolist()
df_sigma_mad

<ipython-input-10-05c68bdc24f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sigma_mad['zphot'] = preds.as_data_frame()['predict'].tolist()


,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC,zphot
ID,,,,,,,,,,,,,,,,,,,,,
SN510924,"[[8.031000000002678, 5.216, 6.285], [28.879000...","[[13.93699999999808, 2.199, 7.08], [44.8870000...","[[0.0, 2.088, 2.24], [13.042999999997846, 1.82...","[[13.953000000001339, 1.99, 5.422], [36.940999...","[0.0, 0.7661717171717413, 1.5323434343434825, ...","([8.031000000002678, 8.70594949495217, 9.38089...","([13.93699999999808, 14.552414141412289, 15.16...","([0.0, 0.7561111111111435, 1.512222222222287, ...","([13.953000000001339, 14.578232323233673, 15.2...","[10.589545779483814, 8.708973184516886, 7.2444...",...,-39.555982,-16.293731,47.963569,-45.448702,-27.989244,-6.281132,Ia,True,0.85640,0.836854
SN511436,"[[1.9689999999973224, 1.07, 3.215], [10.089999...","[[0.01600000000325963, 51.65, 9.372], [2.0, 49...","[[0.0, 35.65, 7.882], [1.9839999999967404, 32....","[[0.9610000000029686, 54.02, 5.43], [9.9409999...","[0.0, 0.8387777777777478, 1.6775555555554955, ...","([1.9689999999973224, 2.6458080808054114, 3.32...","([0.01600000000325963, 0.8545353535386073, 1.6...","([0.0, 0.7076262626262656, 1.4152525252525312,...","([0.9610000000029686, 1.7900707070736157, 2.61...","[9.27014418909458, 8.255517290037698, 7.745205...",...,23.134718,128.783691,-8.731676,12.435225,17.675975,31.130664,Ia,True,0.58313,0.534478
SN511804,"[[2.989000000001397, 3.118, 2.585], [13.055000...","[[0.01599999999598367, 17.51, 3.166], [3.03499...","[[0.0, 1.033, 2.13], [4.082000000002154, 5.482...","[[0.03899999999703141, 19.44, 2.511], [3.06700...","[0.0, 1.1714444444444256, 2.3428888888888513, ...","([2.989000000001397, 4.129939393940805, 5.2708...","([0.01599999999598367, 1.1872828282788337, 2.3...","([0.0, 1.1302525252524929, 2.2605050505049857,...","([0.03899999999703141, 1.208666666663758, 2.37...","[5.178899315805631, 6.259860318410908, 7.21610...",...,18.961628,22.237930,22.428448,16.678827,0.003466,13.356164,Ia,True,0.99576,0.971700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99438,"[[0.0, 2.33, 2.55], [7.894000000000233, 3.805,...","[[0.030999999995401595, 0.006543, 3.683], [7.9...","[[0.014999999999417923, 0.4446, 1.3], [14.0039...","[[0.06199999999807915, 1.497, 4.486], [7.97199...","[0.0, 1.7255757575757058, 3.4511515151514116, ...","([0.0, 1.6948383838383538, 3.3896767676767077,...","([0.030999999995401595, 1.75597979797521, 3.48...","([0.014999999999417923, 1.7298888888882826, 3....","([0.06199999999807915, 1.7869494949475417, 3.5...","[6.789900493579756, 7.110462975693107, 7.57937...",...,-34.417710,32.878025,-33.942007,34.613412,61.324876,-51.210969,Ia,True,0.75966,0.804120
SN99457,"[[0.0, 10.06, 4.768], [4.075000000004366, 4.39...","[[3.0429999999978463, 0.9564, 1.984], [4.10599...","[[3.004000000000815, 1.456, 1.709], [4.0900000...","[[4.137000000002445, 9.726, 4.49], [12.0469999...","[0.0, 1.7667373737373662, 3.5334747474747323, ...","([0.0, 1.7360000000000142, 3.4720000000000284,...","([3.0429999999978463, 4.778717171715048, 6.514...","([3.004000000000815, 4.729858585859406, 6.4557...","([4.137000000002445, 5.861949494951907, 7.5868...","[8.252504299669107, 11.125114322870362, 13.374...",...,9.078515,-15.904951,34.732243,-14.319036,-22.934570,-9.153751,Ia,True,0.80886,0.904851
SN99946,"[[11.961000000002969, 1.526, 5.683], [13.05500...","[[0.01599999999598367, 4.95, 3.147], [3.034999...","[[0.0, 0.4826, 2.129], [2.995999999999185, 0.7...","[[0.03899999999703141, 1.045, 2.483], [12.0389...","[0.0, 1.3626161616161374, 2.725232323232275, 4...","([11.961000000002969, 13.202242424245336, 14.4...","([0.01599999999598367, 1.3782121212081455, 2.7...","([0.0, 1.362181818181816, 2.724363636363632, 4...","([0.03899999999703141, 1.4012222222192594, 2.7...","[3.7693816011787566, 5.06053615747151, 6.78000...",...,183.911735,62.540835,28.895779,-271.089840,54.766397,39.959973,Ia,True,0.40625,0.437865


In [23]:
#pd.concat([X.loc[df_sigma_mad.index],df_sigma_mad['zphot']], axis = 1).to_csv('features_target.csv')
#pd.concat([X.loc[df_sigma_mad.index],df_sigma_mad['zphot']], axis = 1).to_pickle('features_target.pkl')

## Deltas and Sigmas calculus

In [11]:
get_delta_z = lambda zphot, zspec : (zphot - zspec)/(1 + zspec)

In [19]:
delta_z = get_delta_z(df_sigma_mad.zphot.values, df_sigma_mad.REDSHIFT_SPEC.values)
delta_z

array([-0.01052886, -0.03073146, -0.01205545, ...,  0.02526627,
        0.05306721,  0.02248178])

In [20]:
median_delta_z = np.median(delta_z)
median_delta_z
#PREDICTION BIAS

0.0025125752690147193

In [21]:
MAD = np.median(np.abs(delta_z - median_delta_z))
MAD

0.023576926824614566

In [22]:
sigma_MAD = 1.4826*MAD
sigma_MAD

0.034955151710173556

In [23]:
n_fraction = sum(np.abs(delta_z) > 0.05)/len(delta_z)
n_fraction

0.18670886075949367

# Training with different data sizes

In [28]:
df = pd.read_pickle("../../data/structured/df.pkl")
#IA Filter
df = df.loc[df.type_bool == True]
X = df.loc[:,'f1':'REDSHIFT_SPEC']
y = df.loc[:,'REDSHIFT_SPEC']

In [29]:
divisions = [50,125,250,500,1000,1500,2000]
amls = {}
for i in divisions:
    aml = H2OAutoML(max_models=50, seed=1)
    aml.train(x=list(df.loc[:,'f1':'f20'].columns), y='REDSHIFT_SPEC', training_frame=h2o.H2OFrame(X.iloc[0:i]))
    print(aml.leaderboard.as_data_frame(use_pandas=True))
    amls[f'size_{i}'] = aml

Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |███████████
23:56:22.811: Skipping training of model GBM_5_AutoML_20210516_235611 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20210516_235611.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 50.0.


█████████████████████████████████████████████| 100%
                                             model_id  mean_residual_deviance  \
0   DeepLearning_grid__3_AutoML_20210516_235611_mo...                0.011943   
1   DeepLearning_grid__3_AutoML_20210516_235611_mo...                0.012041   
2   DeepLearning_grid__2_AutoML_20210516_235611_mo...                0.012524   
..                                                ...                     ...   
49    XGBoost_grid__1_AutoML_20210516_235611_model_14         

In [47]:
amls

{'size_50': <h2o.automl.autoh2o.H2OAutoML at 0x7f6c30cdbaf0>,
 'size_125': <h2o.automl.autoh2o.H2OAutoML at 0x7f6c306ce1f0>,
 'size_250': <h2o.automl.autoh2o.H2OAutoML at 0x7f6c37090a90>,
 'size_500': <h2o.automl.autoh2o.H2OAutoML at 0x7f6c5a052520>,
 'size_1000': <h2o.automl.autoh2o.H2OAutoML at 0x7f6bfcb7b760>,
 'size_1500': <h2o.automl.autoh2o.H2OAutoML at 0x7f6bfcb7bb50>,
 'size_2000': <h2o.automl.autoh2o.H2OAutoML at 0x7f6c59e7dca0>}

In [54]:
ans = get_metrics_from_batchs(amls, df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%

 26 train/test objects  
 and 24 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

 64 train/test objects  
 and 61 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

 126 train/test objects  
 and 124 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |████████████

In [55]:
ans

{'prediction_bias_size_50': -0.005236580635540106,
 'sigma_MAD_size_50': 0.08901218232558232,
 'n_fraction_size_50': 0.625,
 'prediction_bias_size_125': -0.00378577264047565,
 'sigma_MAD_size_125': 0.049329483194398054,
 'n_fraction_size_125': 0.36065573770491804,
 'prediction_bias_size_250': 0.01279412906296811,
 'sigma_MAD_size_250': 0.057169953608790264,
 'n_fraction_size_250': 0.3790322580645161,
 'prediction_bias_size_500': -0.0035371902158880027,
 'sigma_MAD_size_500': 0.05767373767046159,
 'n_fraction_size_500': 0.42,
 'prediction_bias_size_1000': 0.004510444977170083,
 'sigma_MAD_size_1000': 0.05116786797439472,
 'n_fraction_size_1000': 0.3373493975903614,
 'prediction_bias_size_1500': 0.0003696308490348474,
 'sigma_MAD_size_1500': 0.0471148162178208,
 'n_fraction_size_1500': 0.2868632707774799,
 'prediction_bias_size_2000': 5.539747773482143e-06,
 'sigma_MAD_size_2000': 0.043848679479044694,
 'n_fraction_size_2000': 0.29221435793731043}

In [59]:
get_metrics_from_batchs(amls, df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%

 50 train/test objects  
 and 5028 validation objects


/home/felipe/Documents/Pessoal/projects/redshift_prediction/src/modeling/evaluation.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

 125 train/test objects  
 and 4953 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

 250 train/test objects  
 and 4828 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

 500 train/test objects  
 and 4578 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%

 1000 train/test objects  
 and 4078 validation objects
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████

{'prediction_bias_size_50': -0.003822031978589867,
 'sigma_MAD_size_50': 0.07084421233167937,
 'n_fraction_size_50': 0.4850835322195704,
 'prediction_bias_size_125': -0.009024738337882322,
 'sigma_MAD_size_125': 0.05987677653414336,
 'n_fraction_size_125': 0.4167171411265899,
 'prediction_bias_size_250': -0.0005154952119109519,
 'sigma_MAD_size_250': 0.0574361724519284,
 'n_fraction_size_250': 0.3867025683512842,
 'prediction_bias_size_500': -0.006887987909169122,
 'sigma_MAD_size_500': 0.06114985361263437,
 'n_fraction_size_500': 0.4119702927042377,
 'prediction_bias_size_1000': 0.0019815051499890582,
 'sigma_MAD_size_1000': 0.04930865041962898,
 'n_fraction_size_1000': 0.3182932810201079,
 'prediction_bias_size_1500': -0.0022613612607101253,
 'sigma_MAD_size_1500': 0.04641588172699169,
 'n_fraction_size_1500': 0.2948574622694243,
 'prediction_bias_size_2000': -0.002692721931617943,
 'sigma_MAD_size_2000': 0.04441663812507062,
 'n_fraction_size_2000': 0.2777777777777778}